<h1>Sequence-to-Sequence</h1>

입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델

- 대표적인 예<br>
  - 챗봇
  - 기계번역 <br><br>

입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있음
  - 내용 요약(Text Summarization)
  - STT(Speech to Text)<br>
  등에서 사용

<h1>1. 시퀀스-투-시퀀스(Sequence-to-Sequence)</h1>

seq2seq는 번역기에서 대표적으로 사용되는 모델

앞으로의 설명 방식은 내부가 보이지 않는 커다란 블랙 박스에서 점차적으로 확대해가는 방식으로 설명

여기서 설명하는 내용의 대부분은 RNN 챕터에서 언급한 내용들로 단지 RNN을 어떻게 조립했느냐에 따라서 seq2seq라는 구조가 만들어짐

<img src = "https://wikidocs.net/images/page/24996/%EC%8B%9C%ED%80%80%EC%8A%A4%ED%88%AC%EC%8B%9C%ED%80%80%EC%8A%A4.PNG">

위의 그림은 seq2seq 모델로 만들어진 번역기가  'I am a student'라는 영어 문장을 입력받아서, 'je suis étudiant'라는 프랑스 문장을 출력하는 모습을 보여줌<br><br>

<img src = "https://wikidocs.net/images/page/24996/seq2seq%EB%AA%A8%EB%8D%B811.PNG"><br>

seq2seq는 크게 인코더와 디코더라는 두 개의 모듈로 구성

인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 후 마지막에 모든 단어 정보들을 압축해서 하나의 벡터로 만듦 -> <b>컨텍스트 벡터(context vector)</b>

입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송

디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력<br><br>

<img src = "https://wikidocs.net/images/page/24996/%EC%BB%A8%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B2%A1%ED%84%B0.PNG"><br>

위의 그림에서는 컨텍스트 벡터를 4의 사이즈로 표현했지만, 실제 현업에서 사용되는 seq2seq 모델에서는 보통 수백 이상의 차원을 가짐<br><br>

<img src = "https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG">

인코더와 디코더의 내부를 좀 더 확대

인코더 아키텍쳐와 디코더 아키텍쳐의 내부는 두 개의 RNN 아키텍쳐

입력 문장을 받는 RNN 셀 - 인코더

출력 문장을 출력하는 RNN 셀 - 디코더

인코더의 RNN 셀 - 주황색

디코더의 RNN 셀 - 초록색

성능 문제로 인해 실제로는 바닐라 RNN이 아니라 LSTM 셀 또는 GRU 셀로 구성<br><br>

- 인코더<br>
  입력 문장은 단어 토큰화를 통해 단어 단위로 쪼개지고 단어 토큰 각각은 RNN 셀의 각 시점의 입력이 됨

  인코더 RNN 셀은 모든 단어를 입력받은 뒤에 인코더 RNN 셀의 마지막 시점의 은닉 상태를 디코더 RNN 셀로 넘겨줌 -> <b>컨텍스트 벡터</b>

  컨텍스트 벡터는 디코더 RNN 셀의 첫 번째 은닉 상태에 사용<br><br>

- 디코더<br>
  기본적으로 RNNLM(RNN Language Model)

  디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 \<sos>가 들어감

  디코더는 \<sos>가 입력되면 다음에 등장할 확률이 높은 단어 예측

  첫 번쨰 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je 예측

  첫 번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력

  두 번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보냄

  디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위 반복

  이 행위는 문장의 끝을 의미하는 심볼인 \<eos>가 다음 단어로 예측될 때까지 반복

  지금 설명하는 것은 테스트 과정 동안의 이야기

seq2seq는 훈련 과정과 테스트 과정(또는 실제 번역기를 사람이 쓸 때)의 작동 방식이 상이

훈련 과정에서는 디코더에세 인코더가 보낸 컨텍스트 벡터와 실제 정답인 상황인 \<sos> je suis étudiant를 입력 받았을 때, je suis étudiant \<eos>가 나와야 된다고 정답을 알려주며 훈련

이에 관해서는 뒤에 <b>교사 강요(teacher forcing)</b>를 설명하면서 재언급

반면 테스트 과정에서는 앞서 설명한 과정과 같이 디코더는 오직 컨텍스트 벡터와 \<sos>만을 입력으로 받은 후에 다음에 올 단어를 예측

그 담어를 다음 시점의 RNN 셀의 입력으로 넣는 행위 반복

즉, 앞서 설명한 과정과 위의 그림은 테스트 과정에 해당

이번에는 입, 출력에 쓰이는 단어 토큰들이 있는 부분 확대 <br><br>

<img src = "https://wikidocs.net/images/page/24996/%EB%8B%A8%EC%96%B4%ED%86%A0%ED%81%B0%EB%93%A4%EC%9D%B4.PNG"><br>
기계는 텍스트보다 숫자를 잘 처리

자연어 처리에서 텍스트를 벡터로 바꾸는 방법으로 주로 워드 임베딩 사용
- 워드 임베딩(Word Embedding)을 사용하는 이유는 자연어에서 단어를 컴퓨터가 이해할 수 있는 수치 형태로 표현하면서, 의미와 문맥을 잘 반영하기 위해서

즉, seq2seq에서 사용되는 모든 단어들은 임베딩 벡터로 변환 후 입력으로 사용

위 그림은 모든 단어에 대해서 임베딩 과정을 거치게 하는 단계인 임베딩 층(embedding layer)의 모습을 보여줌<br><br>

<img src = "https://wikidocs.net/images/page/24996/%EC%9E%84%EB%B2%A0%EB%94%A9%EB%B2%A1%ED%84%B0.PNG"><br>
예를 들어 I, am, a, student라는 단어들에 대한 임베딩 벡터는 위와 같은 모습을 가짐

여기서는 그림으로 표현하고자 사이즈를 4로 하였지만, 보통 실제 임베딩 벡터는 수백 개의 차원을 가질 수 있음

RNN 셀에 대해서 확대

이전에 설명하였지만, 하나의 RNN 셀은 각각의 시점(time step)마다 두 개의 입력을 받음<br><br>

<img src = "https://wikidocs.net/images/page/24996/rnn%EA%B7%BC%ED%99%A9.PNG"><br>

현재 시점(time step)을 t라고 할 때, RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듦

이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, 필요없으면 값을 무시할 수 있음

그리고 RNN 셀은 다음 시점에 해당하는 t+1의 RNN 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보냄<br><br>

이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있음

따라서 앞서 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있음<br><br>

디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용

디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 \<sos>로부터, 다음에 등장할 단어를 예측

그리고 이 예측된 단어는 다음 시점인 t+1 RNN에서의 입력값 -> 이 t+1에서의 RNN 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터

즉, 또 다시 다음에 등장할 단어를 예측하게 될 것

디코더가 다음에 등장할 단어를 예측하는 부분을 확대<br><br>

<img src = "https://wikidocs.net/images/page/24996/decodernextwordprediction.PNG"><br>

출력 단어로 나올 수 있는 단어들은 다양한 단어들이 있음

seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야 함

이를 예측하기 위해서 쓸 수 있는 함수 -> <b>소프트맥스 함수</b>

디코더에서 각 시점(time step)의 RNN 셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정<br><br>

지금까지는 가장 기본적인 seq2seq

사실 seq2seq는 어떻게 구현하느냐에 따라서 충분히 더 복잡해질 수 있음

컨텍스트 벡터를 디코더의 초기 은닉 상태로만 사용할 수도 있고, 거기서 더 나아가 컨텍스트 벡터를 디코더가 단어를 예측하는 매 시점마다 하나의 입력으로 사용할 수도 있으며 거기서 더 나아가면 어텐션 메커니즘이라는 방법을 통해 지금 알고있는 컨텍스트 벡터보다 더욱 문맥을 반영할 수 있는 컨텍스트 벡터를 구하여 매 시점마다 하나의 입력으로 사용할 수도 있음

어텐션 메커니즘에 대해서는 다음 챕터에서 배웁니다.


<h1>2. 문자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기</h1>

seq2seq를 이용해서 기계 번역기 구축

<h2>1) 병렬 코퍼스 데이터에 대한 이해와 전처리</h2>

앞서 수행한 태깅 작업의 병렬 데이터와 seq2seq가 사용하는 병렬 데이터는 성격이 상이

태깅 작업의 병렬 데이터는 쌍이 되는 모든 데이터의 길이가 동일했지만 <b><i>여기서는 쌍이 된다고 해서 길이가 같지 않음</b></i><br><br>

제 번역기를 생각해보면 구글 번역기에 '나는 학생이다.'라는 토큰의 개수가 2인 문장을 넣었을 때 'I am a student.'라는 토큰의 개수가 4인 문장이 나오는 것과 같은 이치

seq2seq는 기본적으로 입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있다고 가정

In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [ ]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic'] # lines DataFrame에서 'lic'라는 이름의 열 삭제
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 232736


In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:40000] # 4만개만 저장
lines.sample(10)

,src,tar
23245,They're upstairs.,Elles sont à l'étage.
22298,Look who's there.,Regarde qui est là.
24857,You look worried.,Tu sembles inquiète.
28446,It's the only way.,C'est la seule façon.
28301,It's a girl thing.,C'est un truc de filles.
2820,You're big.,Tu es grande.
1777,Be careful!,Faites attention.
31203,Who wants a drink?,Qui veut une boisson ?
13598,We all laughed.,Nous rîmes tous.
39499,Did I win something?,Ai-je gagné quelque chose ?


위의 테이블은 랜덤으로 선택된 10개의 샘플을 보여줌

번역 문장에 해당되는 프랑스어 데이터는 앞서 배웠듯이 시작을 의미하는 심볼 \<sos>와 종료를 의미하는 심볼 \<eos>를 넣어주어야 함

여기서는 \<sos>와 \<eos> 대신 \t를 시작 심볼, \n을 종료 심볼로 간주하여 추가

In [ ]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
8311,I'll watch it.,\t Je le regarderai. \n
9234,That was fast.,\t Ça a été rapide. \n
38988,You're the teacher.,\t Vous êtes la professeur. \n
2033,I exercise.,\t Je fais de l'exercice. \n
30189,Tom kept coughing.,\t Tom continuait à tousser. \n
957,Did I win?,\t Ai-je gagné ? \n
30928,What a nice house!,\t Quelle belle maison ! \n
9903,We wanted you.,\t Nous vous voulions. \n
31167,Who are these men?,\t Qui sont ces hommes ? \n
34176,I read your report.,\t J'ai lu votre rapport. \n


lines 데이터 프레임의 프랑스어 문장을 수정하고 수정된 데이터프레임에서 무작위로 10개 행의 샘플 표시

대상 프랑스어 문장을 포함하는 lines 데이터프레임의 'tar' 열에 중점

apply() 메서드는 이 열의 각 요소(각 프랑스어 문장)에 함수를 적용하는 데 사용<br><br>

적용되는 함수는 lambda 함수: lambda x : '\t '+ x + ' \n'.

- lambda x: 이것은 'tar' 열의 단일 프랑스어 문장을 나타내는 하나의 인수 x를 사용하는 작고 익명의 함수를 정의
- '\t '+ x + ' \n': 각 문장 x에 대해 수행되는 연산<br>
  문장의 시작 부분에 탭 문자 (\t)와 공백을 추가하고, 문장의 끝 부분에 공백과 개행 문자 (\n)를 추가<br>

기계 번역을 위한 시퀀스-투-시퀀스 모델의 맥락에서 대상 시퀀스의 시작과 끝에 특수 토큰을 추가하는 것이 일반적

이러한 토큰은 모델에 시퀀스의 시작과 끝을 알립니다. 여기서 탭 문자 (\t)는 시퀀스 시작 토큰(<sos>과 같음) 역할을 하고 개행 문자 (\n)는 시퀀스 끝 토큰(<eos>과 같음) 역할

이 줄이 실행된 후에는 lines 데이터프레임의 'tar' 열에 있는 모든 프랑스어 문장에 시작 부분에 탭과 공백이, 끝 부분에 공백과 개행 문자가 추가

랜덤으로 10개의 샘플을 선택하여 출력

프랑스어 데이터에서 시작 심볼과 종료 심볼이 추가된 것 확인

문자 집합 생성. 단어 집합이 아니라 문자 집합이라고 하는 이유는 토큰 단위가 단어가 아니라 문자이기 떄문

In [ ]:
# 문자 집합 구축

src_vocab = set()

for line in lines.src: # 1줄씩 읽음
  for char in line: # 1개의 문자씩 읽음
    src_vocab.add(char)

tar_vocab = set()

for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

src_vocab와 tar_vocab라는 두 개의 빈 집합(set)이 초기화

- 코드는 lines DataFrame의 src 열에 있는 각 줄(문장)을 반복<br>
각 line에 대해 해당 줄의 각 char(문자)를 다시 반복<br>집합의 add() 메서드를 사용하여 현재 char를 src_vocab 집합에 추가<br>만약 해당 문자가 이미 집합에 있다면 다시 추가해도 아무런 영향이 없으므로 src_vocab에는 영어 문장에서 발견된 고유 문자만 포함<br>

마찬가지로, 코드는 lines DataFrame의 tar 열에 대해 동일한 과정을 반복하여 프랑스어 문장에서 발견된 고유 문자들을 포함하는 tar_vocab 집합을 생성 여기에는 이전 단계에서 추가된 특별한 시작(\t) 및 끝(\n) 토큰도 포함

이 코드 블록이 실행된 후,
- src_vocab는 영어 문장에서 발견된 모든 고유 문자들의 집합을 포함
- tar_vocab는 프랑스어 문장에서 발견된 모든 고유 문자들의 집합을 포함


이러한 문자 집합은 나중에 문자와 정수 인덱스 간의 매핑을 생성하는 데 사용

In [ ]:
# 문자 집합의 크기 확인

src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 77
target 문장의 char 집합 : 102


영어와 프랑스어는 각각 79개와 105개의 문자가 존재

이 중에서 인덱스를 임의로 부여하여 일부만 출력

현 상태에서 인덱스를 사용하려고 하면 에러가 나지만 정렬하여 순서를 정해준 뒤에 인덱스를 사용하여 출력해주면 작동

In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


문자 집합에 문자 단위로 저장된 것을 확인

각 문자에 인덱스 부여

In [ ]:
# 각 문자에 인덱스 부여

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 3

인덱스가 부여된 문자 집합으로부터 갖고 있는 훈련 데이터에 정수 인코딩 수행

우선 인코더의 입력이 될 영어 문장 샘플에 대해 정수 인코딩을 수행한 뒤 5개의 샘플 출력

In [ ]:
encoder_input = []

# 1개의 문장
for line in lines.src:

  encoded_line = []

  # 각 줄에서 1개의 char
  for char in line:

    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)

print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


정수 인코딩이 수행된 것을 확인

디코더의 입력이 될 프랑스어 데이터에 대해 정수 인코딩 수행

In [ ]:
decoder_input = []

for line in lines.tar:

  encoded_line = []

  for char in line:
    encoded_line.append(tar_to_index[char])

  decoder_input.append(encoded_line)

print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 46, 50, 3, 4, 3, 2], [1, 3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [1, 3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [1, 3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [1, 3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


정상적으로 정수 인코딩이 수행된 것을 확인

하지만 아직 정수 인코딩을 수행해야 할 데이터가 하나 더 존재

디코더의 예측값과 비교하기 위한 실제값이 필요

그런데 이 실제값에는 시작 심볼에 해당되는 \<sos>가 있을 필요가 없음(이해가 되지 않는다면 이전 페이지의 그림으로 돌아가 Dense와 Softmax 위에 있는 단어들을 다시 보기)

그래서 이번에는 정수 인코딩 과정에서 \<sos>를 제거

즉, 모든 프랑스어 문장의 맨 앞에 붙어있는 '\t'를 제거

In [ ]:
decoder_target = []

for line in lines.tar:

  timestep = 0
  encoded_line = []

  for char in line:

    if timestep > 0:
      encoded_line.append(tar_to_index[char])

    timestep = timestep + 1

  decoder_target.append(encoded_line)

print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 46, 50, 3, 4, 3, 2], [3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


앞서 먼저 만들었던 디코더의 입력값에 해당되는 decoder_input 데이터와 비교하면 decoder_input에서는 모든 문장의 앞에 붙어있던 숫자 1이 decoder_target에서는 제거된 것을 확인

'\t'가 인덱스가 1이므로 정상적으로 제거

모든 데이터에 대해서 정수 인덱스로 변경하였으니 패딩 작업을 수행

패딩을 위해서 영어 문장과 프랑스어 문장 각각에 대해서 가장 길이가 긴 샘플의 길이를 확인

In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 20
target 문장의 최대 길이 : 61


각각 23와 76의 길이를 가짐

이번 병렬 데이터는 영어와 프랑스어의 길이는 하나의 쌍이라고 하더라도 전부 다르므로 패딩을 할 때도 이 두 개의 데이터의 길이를 전부 동일하게 맞춰줄 필요 X

영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞풔 패딩

여기서는 가장 긴 샘플의 길이에 맞춰서 영어 데이터의 샘플은 전부 길이가 23이 되도록 패딩하고, 프랑스어 데이터의 샘플은 전부 길이가 76이 되도록 패딩

In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

모든 값에 대해서 원-핫 인코딩을 수행

문자 단위 번역기므로 워드 임베딩은 별도로 사용되지 않으며, 예측값과의 오차 측정에 사용되는 실제값뿐만 아니라 입력값도 원-핫 벡터를 사용

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

데이터에 대한 전처리가 모두 끝남

 본격적으로 seq2seq 모델을 설계

<h2>2) 교사 강요(Teacher forcing)</h2>

훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용

이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 하기 때문

이런 상황이 반복되면 훈련 시간이 느려짐

만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법 사용 가능

이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법 -> <b><i>교사 강요</i></b>

<h2>3) seq2seq 기계 번역기 훈련</h2>

seq2seq 모델을 설계하고 교사 강요를 사용하여 훈련

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

인코더를 주목해보면 functional API를 사용한다는 것 외에는 앞서 다른 실습에서 본 LSTM 설계와 크게 다르지는 않음

우선 LSTM의 은닉 상태 크기는 256으로 선택

인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True로 설정

인코더에 입력을 넣으면 내부 상태를 리턴<br><br>

LSTM에서 state_h, state_c를 리턴받는데, 이는 각각 LSTM을 설명할 때 언급하였던 배운 은닉 상태와 셀 상태에 해당

앞서 이론을 설명할 때는 셀 상태는 설명에서 생략하고 은닉 상태만 언급하였으나 사실 LSTM은 은닉 상태와 셀 상태라는 두 가지 상태를 가진다는 사실을 기억해야 함

갑자기 어려워진 게 아닙니라 단지 은닉 상태만 전달하는 게 아니라 은닉 상태와 셀 상태 두 가지를 전달한다고 생각

이 두 가지 상태를 encoder_states에 저장

encoder_states를 디코더에 전달하므로서 이 두 가지 상태 모두를 디코더로 전달

이것이 앞서 배운 <b>컨텍스트 벡터</b>

In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

디코더는 인코더의 마지막 은닉 상태를 초기 은닉 상태로 사용합

위에서 initial_state의 인자값으로 encoder_states를 주는 코드가 이에 해당

또한 동일하게 디코더의 은닉 상태 크기도 256으로 부여

디코더도 은닉 상태, 셀 상태를 리턴하기는 하지만 훈련 과정에서는 사용하지 않음

그 후 출력층에 프랑스어의 단어 집합의 크기만큼 뉴런을 배치한 후 소프트맥스 함수를 사용하여 실제값과의 오차 계산

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 1.4072 - val_loss: 0.9439
Epoch 2/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.7518 - val_loss: 0.8086
Epoch 3/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.6443 - val_loss: 0.7326
Epoch 4/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.5870 - val_loss: 0.6875
Epoch 5/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.5411 - val_loss: 0.6383
Epoch 6/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.5075 - val_loss: 0.6064
Epoch 7/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.4793 - val_loss: 0.5840
Epoch 8/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.4573 - val_loss: 0.5593
Epoch 9/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.4363 - val_loss: 0.5417
Epoch 10/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.4204 - val_loss: 0.5243
Epoch 11/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.4061 - val_loss: 0.5141
Epoch 12/40
500/500 ━━━━━━━━━━━━━━━

입력으로는 인코더 입력과 디코더 입력이 들어가고, 디코더의 실제값인 decoder_target도 필요

배치 크기는 64로 하였으며 총 40 에포크를 학습

위에서 설정한 은닉 상태의 크기와 에포크 수는 실제로는 훈련 데이터에 과적합 상태를 불러옴

중간부터 검증 데이터에 대한 오차인 val_loss의 값이 올라가는데, 사실 이번 실습에서는 주어진 데이터의 양과 태스크의 특성으로 인해 훈련 과정에서 훈련 데이터의 정확도와 과적합 방지라는 두 마리 토끼를 동시에 잡기에는 쉽지 않음

여기서는 우선 seq2seq의 메커니즘과 짧은 문장과 긴 문장에 대한 성능 차이에 대한 확인을 중점으로 두고 훈련 데이터에 과적합 된 상태로 동작 단계로 넘어감

<h2>4) seq2seq 기계 번역기 동작</h2>

앞서 seq2seq는 훈련할 때와 동작할 때의 방식이 상이함을 언급

이번에는 입력한 문장에 대해서 기계 번역을 하도록 모델을 조정하고 동작<br><br>

전체적인 번역 동작 단계를 정리하면 아래와 같음
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻음
2. 상태와 \<SOS>에 해당하는 \t를 디코더로 보냄
3. 디코더가 \<EOS>에 해당하는 \n이 나올 때까지 다음 문자를 예측하는 행동을 반복

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

우선 인코더를 정의

encoder_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것들을 재사용

디코더를 설계

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

단어로부터 인덱스를 얻는 것이 아니라 인덱스로부터 단어를 얻을 수 있는 index_to_src와 index_to_tar을 만듦

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Allez-vous ! 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/